In [ ]:
import numpy as np
import pandas as pd
import itertools 
import matplotlib.pyplot as plt

In [ ]:
df_bancolombia = pd.read_csv("bancolombia.csv")
df_bancolombia

,Date,Price,Open,High,Low,Vol.,Change %
0,12/28/2023,"33,200.0","32,880.0","33,200.0","32,800.0",63.78K,0.91%
1,12/27/2023,"32,900.0","32,280.0","32,900.0","32,280.0",127.93K,1.86%
2,12/26/2023,"32,300.0","31,900.0","32,300.0","31,860.0",62.79K,0.31%
3,12/22/2023,"32,200.0","31,880.0","32,300.0","31,640.0",46.85K,-0.31%
4,12/21/2023,"32,300.0","31,800.0","32,300.0","31,540.0",137.91K,1.57%
...,...,...,...,...,...,...,...
967,01/09/2020,"43,300.0","43,500.0","43,500.0","42,500.0",112.47K,-0.37%
968,01/08/2020,"43,460.0","44,480.0","44,480.0","42,580.0",146.91K,-0.78%
969,01/07/2020,"43,800.0","44,180.0","44,180.0","43,540.0",72.68K,-1.13%
970,01/03/2020,"44,300.0","44,360.0","44,360.0","43,900.0",151.96K,-0.18%


# Preprocesamiento

In [ ]:
#Eliminar columnas inútiles
df_bancolombia = df_bancolombia.drop(columns=['Open','High','Low','Vol.','Change %'])

#Convertir los precios de string a float
df_bancolombia['Price'] = df_bancolombia['Price'].str.replace(',', '')
df_bancolombia['Price'] = df_bancolombia['Price'].astype(float)

#Añade la columna Y_n definida en la guía
new_col = [0]
for i in range(1, len(df_bancolombia)):
    new_col.append(float((df_bancolombia['Price'][i]/df_bancolombia['Price'][i-1]) - 1))
df_bancolombia.insert(2, 'Yn', new_col, True)

#Eliminamos primer dato debido a que no se puede calcular Yn
df_bancolombia = df_bancolombia.drop([0])

In [ ]:
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 3,
                       ):
    print(df_bancolombia)

           Date    Price      Yn
1    12/27/2023  32900.0  -0.904
2    12/26/2023  32300.0  -1.824
3    12/22/2023  32200.0  -0.310
4    12/21/2023  32300.0   0.311
5    12/20/2023  31800.0  -1.548
6    12/19/2023  31980.0   0.566
7    12/18/2023  32500.0   1.626
8    12/15/2023  31800.0  -2.154
9    12/14/2023  31900.0   0.314
10   12/13/2023  32000.0   0.313
11   12/12/2023  31240.0  -2.375
12   12/11/2023  30900.0  -1.088
13   12/07/2023  31980.0   3.495
14   12/06/2023  30520.0  -4.565
15   12/05/2023  31800.0   4.194
16   12/04/2023  32620.0   2.579
17   12/01/2023  31400.0  -3.740
18   11/30/2023  31720.0   1.019
19   11/29/2023  30220.0  -4.729
20   11/28/2023  30200.0  -0.066
21   11/27/2023  30200.0   0.000
22   11/24/2023  29640.0  -1.854
23   11/23/2023  29600.0  -0.135
24   11/22/2023  30200.0   2.027
25   11/21/2023  30000.0  -0.662
26   11/20/2023  30000.0   0.000
27   11/17/2023  30200.0   0.667
28   11/16/2023  30180.0  -0.066
29   11/15/2023  30700.0   1.723
30   11/14

# Definición de estados

In [ ]:
new_col = []
l = 20
data = df_bancolombia['Yn']

for i, percentage in enumerate(data):
    if(i >= (l - 1)):
        min = i - l
        
        desviacion = np.std(data[min:i])

        if percentage > (2 * desviacion):
            new_col.append('S3')
        elif percentage > desviacion:
            new_col.append('S2')
        elif percentage > 0:
            new_col.append('S1')
        elif percentage > -(desviacion):
            new_col.append('B1')
        elif percentage > -(2 * desviacion):
            new_col.append('B2')
        else:
            new_col.append('B3')

df_bancolombia = df_bancolombia.drop([i for i in range(1,l)])
df_bancolombia.insert(2, 'Estado', new_col, True)

In [ ]:
df_bancolombia

,Date,Price,Estado,Yn
20,11/28/2023,30200.0,B3,-0.066181
21,11/27/2023,30200.0,B1,0.000000
22,11/24/2023,29640.0,B1,-1.854305
23,11/23/2023,29600.0,B1,-0.134953
24,11/22/2023,30200.0,S1,2.027027
...,...,...,...,...
967,01/09/2020,43300.0,S2,1.547842
968,01/08/2020,43460.0,S1,0.369515
969,01/07/2020,43800.0,S1,0.782329
970,01/03/2020,44300.0,S2,1.141553


In [ ]:
estados_posibles = ['B3','B2','B1','S1','S2','S3']

total_cols = len(estados_posibles)
total_rows = total_cols*total_cols

In [ ]:
estados = np.array(df_bancolombia['Estado'])

markovian_table = np.array(estados)
estados = np.delete(estados, 0)
estados = np.append(estados, 0)
markovian_table = np.vstack([markovian_table,estados])
estados = np.delete(estados, 0)
estados = np.append(estados, 0)
markovian_table = np.vstack([markovian_table,estados])

markovian_table = markovian_table.T
markovian_table = np.delete(markovian_table, [-1, -2], axis=0)
markovian_table 

array([['B3', 'B1', 'B1'],
       ['B1', 'B1', 'B1'],
       ['B1', 'B1', 'S1'],
       ...,
       ['S2', 'S1', 'S1'],
       ['S1', 'S1', 'S2'],
       ['S1', 'S2', 'S1']], dtype=object)

In [ ]:
expected_freq = [[0] * (total_cols + 1) for i in range(total_cols)]

for i in range(len(markovian_table)):
    actual = markovian_table[i][1]
    row_index = estados_posibles.index(actual)
    col_index = estados_posibles.index(markovian_table[i][2])

    expected_freq[row_index][col_index] += 1
    expected_freq[row_index][total_cols] += 1

cols = estados_posibles.copy()
cols.append('Recuento')
expected_freq_df = pd.DataFrame(expected_freq, columns=cols)
cols.pop(-1)
expected_freq_df.insert(0, 'T_Actual', cols)
expected_freq_df

,T_Actual,B3,B2,B1,S1,S2,S3,Recuento
0,B3,4,7,8,6,5,4,34
1,B2,3,15,46,31,13,9,117
2,B1,11,47,118,114,37,8,335
3,S1,4,32,119,99,50,8,312
4,S2,8,15,33,47,9,5,117
5,S3,4,1,10,16,3,1,35


In [ ]:
invalid_counter = 0

for i in range(total_cols):
    for j in range(1,total_cols + 1):
        if expected_freq_df.iat[i, j] < 5:
            invalid_counter += 1
            
cumple_empirica = invalid_counter < 0.25 * (total_cols ** 2)
print(f'Cumple la empírica? RTA: {cumple_empirica}') 

Cumple la empírica? RTA: True


# Modelo - Matriz de transición

In [ ]:
model_freq = [[0] * total_cols for i in range(total_cols)]
total_freq = [0,0,0,0,0,0,0,0]

for i in range(len(markovian_table)):
    actual = markovian_table[i][1]
    row_index = estados_posibles.index(actual)
    col_index = estados_posibles.index(markovian_table[i][2])

    model_freq[row_index][col_index] += 1
    total_freq[row_index] += 1

model_transition_prob_df = pd.DataFrame(model_freq, columns=estados_posibles)
model_transition_prob_df.insert(0, 'Actual', estados_posibles)

for i in range(total_cols):
    for j in range(1,total_cols+1):
        model_transition_prob_df.iat[i, j] = (model_transition_prob_df.iat[i, j]/total_freq[i]).astype(float)

model_transition_prob_df

C:\Users\FAMILIA\AppData\Local\Temp\ipykernel_14684\746005964.py:17: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.11764705882352941' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  model_transition_prob_df.iat[i, j] = (model_transition_prob_df.iat[i, j]/total_freq[i]).astype(float)
C:\Users\FAMILIA\AppData\Local\Temp\ipykernel_14684\746005964.py:17: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.20588235294117646' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  model_transition_prob_df.iat[i, j] = (model_transition_prob_df.iat[i, j]/total_freq[i]).astype(float)
C:\Users\FAMILIA\AppData\Local\Temp\ipykernel_14684\746005964.py:17: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version 

,Actual,B3,B2,B1,S1,S2,S3
0,B3,0.117647,0.205882,0.235294,0.176471,0.147059,0.117647
1,B2,0.025641,0.128205,0.393162,0.264957,0.111111,0.076923
2,B1,0.032836,0.140299,0.352239,0.340299,0.110448,0.023881
3,S1,0.012821,0.102564,0.381410,0.317308,0.160256,0.025641
4,S2,0.068376,0.128205,0.282051,0.401709,0.076923,0.042735
5,S3,0.114286,0.028571,0.285714,0.457143,0.085714,0.028571


# Propiedad Markoviana

In [ ]:
observed_rows = list(itertools.product(estados_posibles,repeat=2))

observed_freq = [[0] * (total_cols + 1) for i in range(total_rows)]

for i in range(len(markovian_table)):
    history = (markovian_table[i][0],markovian_table[i][1])
    row_index = observed_rows.index(history)
    col_index = estados_posibles.index(markovian_table[i][2])

    observed_freq[row_index][col_index] += 1
    observed_freq[row_index][total_cols] += 1

cols = estados_posibles.copy()
cols.append('Recuento')
observed_freq_df = pd.DataFrame(observed_freq, columns=cols)
observed_freq_df.insert(0, 'History', observed_rows)
observed_freq_df

,History,B3,B2,B1,S1,S2,S3,Recuento
0,"(B3, B3)",0,2,0,1,1,0,4
1,"(B3, B2)",2,0,2,1,2,0,7
2,"(B3, B1)",0,3,5,1,0,0,9
3,"(B3, S1)",0,1,3,1,1,0,6
4,"(B3, S2)",0,1,1,3,0,0,5
5,"(B3, S3)",0,1,2,1,0,0,4
6,"(B2, B3)",0,3,0,0,0,0,3
7,"(B2, B2)",0,1,7,5,1,1,15
8,"(B2, B1)",3,9,15,9,6,4,46
9,"(B2, S1)",1,3,10,9,6,2,31


In [ ]:
observed_prob_df = observed_freq_df.copy()
for i in range(total_rows):
    for j in range(1,total_cols+1):
        if observed_prob_df.iat[i, total_cols+1] == 0:
            observed_prob_df.iat[i, j] = 0
        else:
            observed_prob_df.iat[i, j] = (observed_prob_df.iat[i, j]/observed_prob_df.iat[i, total_cols+1]).astype(float)

observed_prob_df = observed_prob_df.drop(columns=['Recuento'], axis=1)
observed_prob_df

C:\Users\FAMILIA\AppData\Local\Temp\ipykernel_14684\1372542959.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  observed_prob_df.iat[i, j] = (observed_prob_df.iat[i, j]/observed_prob_df.iat[i, total_cols+1]).astype(float)
C:\Users\FAMILIA\AppData\Local\Temp\ipykernel_14684\1372542959.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.25' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  observed_prob_df.iat[i, j] = (observed_prob_df.iat[i, j]/observed_prob_df.iat[i, total_cols+1]).astype(float)
C:\Users\FAMILIA\AppData\Local\Temp\ipykernel_14684\1372542959.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Valu

,History,B3,B2,B1,S1,S2,S3
0,"(B3, B3)",0.000000,0.500000,0.000000,0.250000,0.250000,0.000000
1,"(B3, B2)",0.285714,0.000000,0.285714,0.142857,0.285714,0.000000
2,"(B3, B1)",0.000000,0.333333,0.555556,0.111111,0.000000,0.000000
3,"(B3, S1)",0.000000,0.166667,0.500000,0.166667,0.166667,0.000000
4,"(B3, S2)",0.000000,0.200000,0.200000,0.600000,0.000000,0.000000
5,"(B3, S3)",0.000000,0.250000,0.500000,0.250000,0.000000,0.000000
6,"(B2, B3)",0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
7,"(B2, B2)",0.000000,0.066667,0.466667,0.333333,0.066667,0.066667
8,"(B2, B1)",0.065217,0.195652,0.326087,0.195652,0.130435,0.086957
9,"(B2, S1)",0.032258,0.096774,0.322581,0.290323,0.193548,0.064516


In [ ]:
expected_freq = [[0] * (total_cols + 1) for i in range(total_cols)]

for i in range(len(markovian_table)):
    actual = markovian_table[i][1]
    row_index = estados_posibles.index(actual)
    col_index = estados_posibles.index(markovian_table[i][2])

    expected_freq[row_index][col_index] += 1
    expected_freq[row_index][total_cols] += 1

cols = estados_posibles.copy()
cols.append('Recuento')
expected_freq_df = pd.DataFrame(expected_freq, columns=cols)
cols.pop(-1)
expected_freq_df.insert(0, 'History', cols)
expected_freq_df

,History,B3,B2,B1,S1,S2,S3,Recuento
0,B3,4,7,8,6,5,4,34
1,B2,3,15,46,31,13,9,117
2,B1,11,47,118,114,37,8,335
3,S1,4,32,119,99,50,8,312
4,S2,8,15,33,47,9,5,117
5,S3,4,1,10,16,3,1,35


In [ ]:
expected_prob_df = expected_freq_df.copy()
for i in range(total_cols):
    for j in range(1,total_cols+1):
        expected_prob_df.iat[i, j] = (expected_prob_df.iat[i, j]/expected_prob_df.iat[i, total_cols+1]).astype(float)

expected_prob_df = expected_prob_df.drop(columns=['Recuento'], axis=1)
expected_prob_df

C:\Users\FAMILIA\AppData\Local\Temp\ipykernel_14684\1491194249.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.11764705882352941' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  expected_prob_df.iat[i, j] = (expected_prob_df.iat[i, j]/expected_prob_df.iat[i, total_cols+1]).astype(float)
C:\Users\FAMILIA\AppData\Local\Temp\ipykernel_14684\1491194249.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.20588235294117646' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  expected_prob_df.iat[i, j] = (expected_prob_df.iat[i, j]/expected_prob_df.iat[i, total_cols+1]).astype(float)
C:\Users\FAMILIA\AppData\Local\Temp\ipykernel_14684\1491194249.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a

,History,B3,B2,B1,S1,S2,S3
0,B3,0.117647,0.205882,0.235294,0.176471,0.147059,0.117647
1,B2,0.025641,0.128205,0.393162,0.264957,0.111111,0.076923
2,B1,0.032836,0.140299,0.352239,0.340299,0.110448,0.023881
3,S1,0.012821,0.102564,0.381410,0.317308,0.160256,0.025641
4,S2,0.068376,0.128205,0.282051,0.401709,0.076923,0.042735
5,S3,0.114286,0.028571,0.285714,0.457143,0.085714,0.028571


In [ ]:
chi_values = []
k = 0

for i in range(total_rows):
    for j in range(1,total_cols+1):
        if expected_prob_df.iat[k,j] == 0:
            chi_parcial = (0)
        else: 
            chi_parcial = observed_freq_df.iat[i,total_cols+1] * ((observed_prob_df.iat[i,j] - expected_prob_df.iat[k,j]) ** 2) / expected_prob_df.iat[k,j]

        chi_values.append(chi_parcial)

    k += 1
    if k == total_cols:
        k = 0

chi_observado = sum(chi_values)
chi_observado

184.05140877540757

In [ ]:
from scipy.stats.distributions import chi2

alpha = 0.01
rows = total_rows
cols = total_cols
grados = (rows - 1)*(cols - 1)

chi_teorico = chi2.ppf(1 - alpha, df=grados)
chi_teorico

221.4383745662851

In [ ]:
is_not_markovian = chi_observado >= chi_teorico
print(f'Se rechaza la cadena (no es markoviana)? RTA: {is_not_markovian}')

Se rechaza la cadena (no es markoviana)? RTA: False


# Propiedad Estacionaria

Definiremos los periodos de tiempo como años

In [ ]:
new_col = []
for date in df_bancolombia['Date']:
    if '2020' in date:
        new_col.append(1)
    elif '2021' in date:
        new_col.append(2)
    elif '2022' in date:
        new_col.append(3)
    else:
        new_col.append(4)
df_bancolombia.insert(2, 'Periodo', new_col, True)

df_bancolombia

,Date,Price,Periodo,Estado,Yn
20,11/28/2023,30200.0,4,B3,-0.066181
21,11/27/2023,30200.0,4,B1,0.000000
22,11/24/2023,29640.0,4,B1,-1.854305
23,11/23/2023,29600.0,4,B1,-0.134953
24,11/22/2023,30200.0,4,S1,2.027027
...,...,...,...,...,...
967,01/09/2020,43300.0,1,S2,1.547842
968,01/08/2020,43460.0,1,S1,0.369515
969,01/07/2020,43800.0,1,S1,0.782329
970,01/03/2020,44300.0,1,S2,1.141553


In [ ]:
periodos = np.array(df_bancolombia['Periodo'])
estados = np.array(df_bancolombia['Estado'])

stationary_table = np.array(periodos)
stationary_table = np.vstack([stationary_table, estados])
estados = np.delete(estados, 0)
estados = np.append(estados, 0)
stationary_table = np.vstack([stationary_table, estados])

stationary_table = stationary_table.T
stationary_table = np.delete(stationary_table, -1, axis=0)
stationary_table 

array([[4, 'B3', 'B1'],
       [4, 'B1', 'B1'],
       [4, 'B1', 'B1'],
       ...,
       [1, 'S1', 'S1'],
       [1, 'S1', 'S2'],
       [1, 'S2', 'S1']], dtype=object)

In [ ]:
observed_rows = list(itertools.product([1,2,3,4], estados_posibles))

observed_freq = [[0] * (total_cols + 1) for i in range(len(observed_rows))]

for i in range(len(markovian_table)):
    history = (stationary_table[i][0], stationary_table[i][1])
    row_index = observed_rows.index(history)
    col_index = estados_posibles.index(stationary_table[i][2])

    observed_freq[row_index][col_index] += 1
    observed_freq[row_index][total_cols] += 1

cols = estados_posibles.copy()
cols.append('Recuento')
observed_freq_stat_df = pd.DataFrame(observed_freq, columns=cols)
observed_freq_stat_df.insert(0, 'History', observed_rows)
observed_freq_stat_df

,History,B3,B2,B1,S1,S2,S3,Recuento
0,"(1, B3)",1,0,3,2,1,0,7
1,"(1, B2)",0,5,12,10,2,1,30
2,"(1, B1)",1,11,31,30,8,2,83
3,"(1, S1)",3,12,25,26,16,3,85
4,"(1, S2)",1,2,10,12,0,2,27
5,"(1, S3)",1,0,2,5,0,0,8
6,"(2, B3)",1,4,1,1,2,1,10
7,"(2, B2)",2,2,11,7,7,1,30
8,"(2, B1)",3,14,26,27,9,4,83
9,"(2, S1)",0,6,35,32,10,1,84


In [ ]:
observed_prob_stat_df = observed_freq_stat_df.copy()
for i in range(len(observed_rows)):
    for j in range(1,total_cols+1):
        if observed_prob_stat_df.iat[i, total_cols+1] == 0:
            observed_prob_stat_df.iat[i, j] = 0
        else:
            observed_prob_stat_df.iat[i, j] = (observed_prob_stat_df.iat[i, j]/observed_prob_stat_df.iat[i, total_cols+1]).astype(float)

observed_prob_stat_df = observed_prob_stat_df.drop(columns=['Recuento'], axis=1)
observed_prob_stat_df

C:\Users\FAMILIA\AppData\Local\Temp\ipykernel_14684\1821551178.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.14285714285714285' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  observed_prob_stat_df.iat[i, j] = (observed_prob_stat_df.iat[i, j]/observed_prob_stat_df.iat[i, total_cols+1]).astype(float)
C:\Users\FAMILIA\AppData\Local\Temp\ipykernel_14684\1821551178.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.42857142857142855' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  observed_prob_stat_df.iat[i, j] = (observed_prob_stat_df.iat[i, j]/observed_prob_stat_df.iat[i, total_cols+1]).astype(float)
C:\Users\FAMILIA\AppData\Local\Temp\ipykernel_14684\1821551178.py:7: FutureWarning: Setting an item of incompatible dtype is deprecate

,History,B3,B2,B1,S1,S2,S3
0,"(1, B3)",0.142857,0.000000,0.428571,0.285714,0.142857,0.000000
1,"(1, B2)",0.000000,0.166667,0.400000,0.333333,0.066667,0.033333
2,"(1, B1)",0.012048,0.132530,0.373494,0.361446,0.096386,0.024096
3,"(1, S1)",0.035294,0.141176,0.294118,0.305882,0.188235,0.035294
4,"(1, S2)",0.037037,0.074074,0.370370,0.444444,0.000000,0.074074
5,"(1, S3)",0.125000,0.000000,0.250000,0.625000,0.000000,0.000000
6,"(2, B3)",0.100000,0.400000,0.100000,0.100000,0.200000,0.100000
7,"(2, B2)",0.066667,0.066667,0.366667,0.233333,0.233333,0.033333
8,"(2, B1)",0.036145,0.168675,0.313253,0.325301,0.108434,0.048193
9,"(2, S1)",0.000000,0.071429,0.416667,0.380952,0.119048,0.011905


In [ ]:
expected_prob_df

,History,B3,B2,B1,S1,S2,S3
0,B3,0.117647,0.205882,0.235294,0.176471,0.147059,0.117647
1,B2,0.025641,0.128205,0.393162,0.264957,0.111111,0.076923
2,B1,0.032836,0.140299,0.352239,0.340299,0.110448,0.023881
3,S1,0.012821,0.102564,0.381410,0.317308,0.160256,0.025641
4,S2,0.068376,0.128205,0.282051,0.401709,0.076923,0.042735
5,S3,0.114286,0.028571,0.285714,0.457143,0.085714,0.028571


In [ ]:
chi_values = []
k = 0

for i in range(len(observed_rows)):
    for j in range(1,total_cols+1):
        if expected_prob_df.iat[k,j] == 0:
            chi_parcial = 0
        else: 
            chi_parcial = observed_freq_stat_df.iat[i,total_cols+1] * ((observed_prob_stat_df.iat[i,j] - expected_prob_df.iat[k,j]) ** 2) / expected_prob_df.iat[k,j]

        chi_values.append(chi_parcial)

    k += 1
    if k == total_cols:
        k = 0

chi_stat_observado = sum(chi_values)
chi_stat_observado

83.58180903134445

In [ ]:
is_not_stationary = chi_stat_observado >= chi_teorico
print(f'La cadena no cumple la propiedad estacionaria? RTA: {is_not_stationary}')

La cadena no cumple la propiedad estacionaria? RTA: False
